In [1]:
#pip install numpy

In [2]:
from copy import deepcopy
import random
import numpy as np
from abc import ABC, abstractmethod

In [3]:
UNOCCUPIED = 0
WHITE = 1
BLACK = 2

def color_to_string(color):
    c = ' · '
    if color == WHITE:
        c = ' x '
    if color == BLACK:
        c = ' o '
    return c

class Board:
    def __init__(self):
        self.board = [np.uint64(0), np.uint64(0)]
        self.set(3, 3, WHITE)
        self.set(4, 4, WHITE)
        self.set(3, 4, BLACK)
        self.set(4, 3, BLACK)

    def set(self, x, y, color):
        index = -1
        if color == WHITE:
            index = 0
        else:
            index = 1

        if index != -1:
            mask = np.uint64(1)<<np.uint64(y * 8 + x)
            self.board[index] = np.bitwise_or(self.board[index], mask)

    def clear(self, x, y):
        mask = np.uint64(1)<<np.uint64(y * 8 + x)
        self.board[0] = np.bitwise_and(self.board[0], ~mask)
        self.board[1] = np.bitwise_and(self.board[1], ~mask)  


    def get(self, x, y):
        mask = np.uint64(1)<<np.uint64(y * 8 + x)
        if np.bitwise_and(self.board[0], mask) > 0:
            return WHITE
        
        if np.bitwise_and(self.board[1], mask) > 0:
            return BLACK

        return UNOCCUPIED

    def _countBits(self, n):
        count = np.uint64(0)
        while (n):
           count += np.bitwise_and(n, np.uint64(1))
           n = n >> np.uint64(1)
        return np.int64(count).item()

    def toInputVector(self):
        v = np.zeros(128)
        i = 0
        for j in range(2):
            piecesWord = self.board[j]
            while (piecesWord):
                v[i] = np.bitwise_and(piecesWord, np.uint64(1))
                piecesWord = piecesWord >> np.uint64(1)
                i += 1
        return v[np.newaxis]

    def count(self):
        whites = self.board[0]
        blacks = self.board[1]
        return (self._countBits(whites), self._countBits(blacks))

    def print(self):
        print('   a  b  c  d  e  f  g  h')
        for y in range(8):
            print('{} '.format(y+1), end='')
            for x in range(8):
                color = self.get(x, y)
                c = color_to_string(color)
                print(c, end = '')
            print('')

               
        

    
    

In [4]:
def other_color(color):
    if color == WHITE:
        return BLACK
    if color == BLACK:
        return WHITE

def make_move(x, y, color, b):
    if x < 0 or x > 7 or y < 0 or y > 7 or b.get(x, y) != UNOCCUPIED:
        return 0

    dx = [-1, 0, 1, -1, 1, -1, 0, 1]
    dy = [-1, -1, -1, 0, 0, 1, 1, 1]

    flipped = 0
    otherColor = other_color(color)

    for d in range(8):
        cx = x + dx[d]
        cy = y + dy[d]

        hasFoundMyOwnAtEnd = False
        while cx >= 0 and cy >= 0 and cx < 8 and cy < 8 and b.get(cx, cy) == otherColor:
            cx += dx[d]
            cy += dy[d]
            hasFoundMyOwnAtEnd = True

        if cx >= 0 and cy >= 0 and cx < 8 and cy < 8 and b.get(cx, cy) == color and hasFoundMyOwnAtEnd:
            # this is a valid move, we found our own color after
            # step all the way back to start and flip all of the other color
            cx -= dx[d]
            cy -= dy[d]
            while cx != x or cy != y:
                b.clear(cx, cy)
                b.set(cx, cy, color)
                cx -= dx[d]
                cy -= dy[d]
                flipped += 1
    
    if flipped > 0:
        b.set(x, y, color)

    return flipped

def to_coord(x, y):
    return '{}{}'.format("abcdefgh"[x], y+1)
    
def get_possible_boards(b, color):
    boards = []
    for y in range(8):
        for x in range(8):
            clone = deepcopy(b)
            flipped = make_move(x, y, color, clone)
            if flipped > 0:
                boards.append((clone, flipped, to_coord(x, y)))
    return sorted(boards, key=lambda a: a[1], reverse = True)

def game_over(b):
    return len(get_possible_boards(b, WHITE)) == 0 and len(get_possible_boards(b, BLACK)) == 0

    
b = Board()
flipped = make_move(3, 2, BLACK, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(2, 4, WHITE, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(1, 5, BLACK, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(1, 4, WHITE, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(1, 6, WHITE, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(0, 6, BLACK, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped = make_move(5, 5, BLACK, b)
print('flipped {} tiles'.format(flipped))
b.print()

flipped 1 tiles
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  o  ·  ·  ·  · 
4  ·  ·  ·  o  o  ·  ·  · 
5  ·  ·  ·  o  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 
flipped 1 tiles
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  o  ·  ·  ·  · 
4  ·  ·  ·  o  o  ·  ·  · 
5  ·  ·  x  x  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 
flipped 1 tiles
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  o  ·  ·  ·  · 
4  ·  ·  ·  o  o  ·  ·  · 
5  ·  ·  o  x  x  ·  ·  · 
6  ·  o  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 
flipped 1 tiles
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  o  ·  ·  ·  · 
4  ·  ·  ·  o  o  ·  ·  · 
5  ·  x  x  x  x  ·  ·  · 
6  ·  o  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·

In [5]:
from typing import Tuple

class Player(ABC):
    
    def __init__(self, c):
        self.mycolor = c
        self.verbose = True

    @abstractmethod
    def make_move(self, b) -> Tuple[Board, str]:
        pass

    def reset(self):
        pass

    def contemplate(self, b, game_over=False):
        pass

    def setVerbosity(self, verbose):
        self.verbose = verbose

In [6]:
class RandomPlayer(Player):  
    def __init__(self, c):
        super().__init__(c)

    def make_move(self, b) -> Tuple[Board, str]:
        boards = get_possible_boards(b, self.mycolor)
        if len(boards) > 0:
            b, _, coord = random.choice(boards)
            return (b, coord)
        return (None, '')

In [7]:
class GreedyPlayer(Player):
    def __init__(self, c):
        super().__init__(c)

    def make_move(self, b) -> Tuple[Board, str]:
        boards = get_possible_boards(b, self.mycolor)
        if len(boards) > 0:
            b, _, coord = boards[0]
            return (b, coord)
        return (None, '')

In [8]:
class InteractivePlayer(Player):
    def __init__(self, c):
        super().__init__(c)

    def make_move(self, b) -> Tuple[Board, str]:
        boards = get_possible_boards(b, self.mycolor)
        if len(boards) > 0:
            done = False
            while not done:
                b.print()
                move = input("enter your move: ")
                if move == 'q':
                    done = True
                if len(move) == 2:
                    col = move[0]
                    row = move[1]
                    x = ord(col) - ord('a')
                    y = ord(row) - ord('0') - 1 
                    if make_move(x, y, self.mycolor, b) > 0:
                        return (b, move)
                    else:
                        print("not a valid move")
                else:
                    print("not a valid move")
        return (None, '')

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import InputLayer
import tensorflow as tf

def get_model(n_inputs, n_outputs, n_hidden):
    model = Sequential()
    model.add(InputLayer(input_shape=(n_inputs, ), name='input_layer'))
    model.add(Dense(n_hidden, kernel_initializer='he_uniform', activation='sigmoid', name='hidden_layer'))
    model.add(Dense(n_outputs, name='output_layer', activation='sigmoid'))
    model.compile(loss='mae', optimizer='adam')
    return model

In [10]:
import pickle

class NN:
    def __init__(self, params):
        self.model = get_model(**params)

    def evaluate(self, board) -> float:
        X_values = board.toInputVector()
        return self.model.predict(X_values, verbose=0)[0][0]

    def trainable_variables(self):
        return self.model.trainable_variables


ALPHA = 0.1
LAMBDA = 0.7

class TDPlayer(Player):
    def __init__(self, c, model = None):
        super().__init__(c)
        params = {'n_inputs': 128, 'n_hidden': 60, 'n_outputs': 1}
        if model is not None:
            self.model = model
        else:
            self.model = NN(params)

        self.predictions = [] # predictions holds tuples of (board, prediction), from first move to last
        self.t = 0
        self.grads = []
        self.gamesPlayed = 0

    def save(self, fname):
        with open(fname, 'wb') as file:
            persistedState = {'gamesPlayed': self.gamesPlayed, 'model': self.model}
            pickle.dump(persistedState, file)

    def load(self, fname):
        with open(fname, 'rb') as file:
            self.reset()
            persistedState = pickle.load(file)
            print(persistedState)
            self.gamesPlayed = persistedState['gamesPlayed']
            self.model = persistedState['model']

    def reset(self):
        self.predictions = []
        self.t = 1
        self.grads = []
        self.gamesPlayed += 1

    def make_move(self, b) -> Tuple[Board, str]:
        boards = get_possible_boards(b, self.mycolor)
        bestCandidate = None
        bestCoord = ''
        bestSoFar = -np.Inf

        if len(boards) > 0:
           
            for bs in boards:
                candidate, _, coord = bs
                p = self.model.evaluate(candidate)
                
                if self.mycolor == BLACK:
                    p = 1 - p

                if p > bestSoFar:
                    bestCandidate = candidate
                    bestCoord = coord
                    bestSoFar = p

        return (bestCandidate, bestCoord)

    def contemplate(self, b, game_over=False):
        # adjust the weights according to TD(λ)
        X_values = b.toInputVector()
        with tf.GradientTape() as tape:
            p = self.model.model(X_values)

        trainable_vars = self.model.trainable_variables()
        gradients = tape.gradient(p, trainable_vars)
       
        self.predictions.append(p[0][0])
        self.grads.append(gradients)

        t = self.t

        if len(self.predictions) > 1:
            # update weights in all layers (len(trainable_vars) == len(grads), but each trainable_vars[i] is a tensor but grads[i] is a list)
            for layer in range(len(trainable_vars)):
                layer_trainable_vars = trainable_vars[layer]

                weighted_gradient_sum = tf.zeros(shape=layer_trainable_vars.shape)
                for k in range(1, t):
                    layer_gradients_k = tf.Variable(self.grads[k-1][layer], shape=layer_trainable_vars.shape)
                    weighted_gradient_sum = tf.math.add(
                        weighted_gradient_sum, 
                        tf.math.scalar_mul(LAMBDA ** (t - k), layer_gradients_k))

                Y_t_plus_1 = self.predictions[t-1] #this would be the real outcome if we reach end of game
                if game_over:
                    # count final standing
                    whites, blacks = b.count()
                    if whites > blacks:
                        Y_t_plus_1 = 1.0
                    else: 
                        if whites == blacks:
                            Y_t_plus_1 = 0.5
                        else:
                            Y_t_plus_1 = 0.0


                Y_t = self.predictions[t-2]
                reward = ALPHA * (Y_t_plus_1 - Y_t)
                weight_delta = tf.math.scalar_mul(reward, weighted_gradient_sum)
                layer_trainable_vars.assign_add(weight_delta) 

        self.t += 1

In [11]:
def play_game(blackPlayer, whitePlayer, verbose=True, allowContemplation=True):
    b = Board()
    blackPlayer.reset()
    whitePlayer.reset()
    blackPlayer.setVerbosity(verbose)
    whitePlayer.setVerbosity(verbose)

    coords = []
    while True:
        # black player makes a move (if possible)
        blackMadeMove = False
        move, coord = blackPlayer.make_move(b)
        if move is not None:
            b = move
            blackMadeMove = True
            coords.append(coord)
            if verbose:
                print('black plays {}'.format(coord))
                b.print()
                print("---")
            if allowContemplation:
                blackPlayer.contemplate(b)
        

        # white player makes a move (if possible)
        whiteMadeMove = False
        move, coord = whitePlayer.make_move(b)
        if move is not None:
            b = move
            whiteMadeMove = True
            coords.append(coord)
            if verbose:
                print('white plays {}'.format(coord))
                b.print()
                print('')
                print("---")
            if allowContemplation:
                whitePlayer.contemplate(b)

        # if no player could make a move we are done
        if not blackMadeMove and not whiteMadeMove:
            if verbose:
                print("no player can make a move - game over")
            break

    # count final standing
    whites, blacks = b.count()
    if allowContemplation:
        blackPlayer.contemplate(b, True)
        whitePlayer.contemplate(b, True)
    return (whites, blacks, coords)

In [16]:
# self training
F_NAME = 'tdPlayer.pickle'

# initialize white player
whitePlayer = TDPlayer(WHITE)
# load model from file 
whitePlayer.load(F_NAME)

# initialize black player with white's model (so they use the same model)
blackPlayer = TDPlayer(BLACK, model=whitePlayer.model)

N_EPOCHS = 200
N_GAMES_PER_EPOCH = 500
N_TESTS_PER_EPOCH = 100

for epoch in range(N_EPOCHS):
    print('training epoch {}'.format(epoch+1))
    for g in range(N_GAMES_PER_EPOCH):
        if g % 5 == 0:
            print('.', end = '')
        play_game(blackPlayer, whitePlayer, verbose=False)

    
    whiteWins = 0
    blackWins = 0
    print('\ntesting after epoch {}'.format(epoch+1))
    for t in range(N_TESTS_PER_EPOCH):
        if t % 5 == 0:
            print('.', end = '')
        whites, blacks, coords = play_game(RandomPlayer(BLACK), whitePlayer, verbose=False)
        if whites > blacks:
            whiteWins += 1
        else: 
            if blacks > whites:
                blackWins += 1

    print('\ntest results after epoch {}'.format(epoch+1))
    print('white wins: {}, black wins: {} ({}%)'.format(whiteWins, blackWins, round(100.0 * whiteWins / N_TESTS_PER_EPOCH, 2)))
    whitePlayer.save(F_NAME)

{'gamesPlayed': 194500, 'model': <__main__.NN object at 0xe0d2d32e0>}
training epoch 1
.

2023-03-23 08:12:20.720066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...................................................................................................
testing after epoch 1
....................


test results after epoch 1
white wins: 86, black wins: 9 (86.0%)
INFO:tensorflow:Assets written to: ram://16077d31-cbd7-4107-b6c9-e1476192f560/assets


INFO:tensorflow:Assets written to: ram://16077d31-cbd7-4107-b6c9-e1476192f560/assets


...

INFO:tensorflow:Assets written to: ram://4003afb4-ed5c-48a5-b1d9-a64b4de6d5d9/assets


training epoch 16
....................................................................................................
testing after epoch 16
....................


test results after epoch 16
white wins: 86, black wins: 11 (86.0%)
INFO:tensorflow:Assets written to: ram://8c24f087-f995-41a4-9438-ac8d2def605a/assets


INFO:tensorflow:Assets written to: ram://8c24f087-f995-41a4-9438-ac8d2def605a/assets


training epoch 17
..............................................................................

In [15]:
# self training
F_NAME = 'tdPlayer.pickle'

# initialize white player
whitePlayer = TDPlayer(WHITE)
# load model from file 
whitePlayer.load(F_NAME)

whites, blacks, coords = play_game(InteractivePlayer(BLACK), whitePlayer, verbose = True, allowContemplation = True)

if whites > blacks:
    print("WHITE WON!")
else: 
    if blacks > whites:
        print("BLACK WON!")
    else:
        print("THERE WAS A TIE!")

print('({} whites vs {} blacks)'.format(whites, blacks))

{'gamesPlayed': 194500, 'model': <__main__.NN object at 0xe0bef4bb0>}
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  ·  ·  ·  ·  · 
4  ·  ·  ·  x  o  ·  ·  · 
5  ·  ·  ·  o  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 


2023-03-23 07:57:50.665651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


black plays d3
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  ·  o  ·  ·  ·  · 
4  ·  ·  ·  o  o  ·  ·  · 
5  ·  ·  ·  o  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 
---
white plays c3
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  x  o  ·  ·  ·  · 
4  ·  ·  ·  x  o  ·  ·  · 
5  ·  ·  ·  o  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 

---
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  x  o  ·  ·  ·  · 
4  ·  ·  ·  x  o  ·  ·  · 
5  ·  ·  ·  o  x  ·  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  ·  ·  ·  ·  ·  ·  ·  · 
black plays f5
   a  b  c  d  e  f  g  h
1  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
3  ·  ·  x  o  ·  ·  ·  · 
4  ·  ·  ·  x  o  ·  ·  · 
5  ·  ·  ·  o  o  o  ·  · 
6  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
8  · 